# Pipeline de Súmulas do STJ

Notebook para extrair súmulas do site do STJ, pré-processar os verbetes, gerar embeddings com SentenceTransformers e indexá-los com FAISS para busca semântica.

Este notebook contém passos para coleta (web scraping), preparação de dados, geração de embeddings, construção do índice e exemplos de consulta.

## Coleta (web scraping)

Este bloco utiliza Selenium para navegar no site do STJ e coletar número e verbete de cada súmula. Os resultados são salvos em `sumulas_metadata.json`.

In [1]:
import json
import re
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

/home/arthur-silva-dantas/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

In [3]:
driver.get("https://scon.stj.jus.br/SCON/sumstj/toc.jsp?documentosSelecionadosParaPDF=&numDocsPagina=100&tipo_visualizacao=&data=&p=true&b=SUMU&thesaurus=JURIDICO&i=1&l=100&ordenacao=-%40NUM&operador=e")

sumulas = []
while True:
    for grid in driver.find_elements(By.CLASS_NAME, "gridSumula"):
        try:
            num = grid.find_element(By.CLASS_NAME, "numeroSumula").text
            verbete = grid.find_element(By.CLASS_NAME, "blocoVerbete").text
            sumulas.append({"numero": num, "verbete": verbete})
        except Exception:
            continue
    try:
        botao_proxima = driver.find_element(By.CLASS_NAME, "iconeProximaPagina")
        botao_proxima.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(grid))
    except Exception:
        break

with open("sumulas_metadata.json", "w", encoding="utf-8") as f:
    json.dump(sumulas, f, ensure_ascii=False, indent=2)


## Geração de embeddings e indexação

Aqui carregamos os verbetes extraídos, geramos embeddings usando `sentence-transformers` e criamos um índice FAISS (IndexFlatIP). Os vetores são normalizados com L2 antes de gravar o índice.

In [4]:
with open("sumulas_metadata.json", "r", encoding="utf-8") as f:
    sumulas = json.load(f)

verbetes = [s["verbete"] for s in sumulas]
embeddings = model.encode(verbetes, convert_to_numpy=True)
embeddings = np.asarray(embeddings, dtype=np.float32)
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

faiss.write_index(index, "sumulas_index.faiss")

## Pré-processamento e funções utilitárias

Funções para normalizar e limpar texto jurídico (`preprocessar_texto_juridico`), gerar embeddings localmente (`gerar_embedding_local`) e realizar consultas semânticas (`buscar_sumulas`).

In [5]:
def preprocessar_texto_juridico(texto):
    if not texto:
        return ''
    
    texto = texto.lower()
    texto = re.sub(r'\d{7}-\d{2}\.\d{4}\.\d{1}\.\d{2}\.\d{4}', '', texto)
    texto = re.sub(r'\b(art\.|artigo)\s*\d+', 'artigo', texto)
    texto = re.sub(r'\b(inc\.|inciso)\s*[ivxlcdm]+', 'inciso', texto)
    texto = re.sub(r'\b(par\.|parágrafo)\s*\d+', 'parágrafo', texto)
    texto = re.sub(r'\bcpc/?\d{4}\b', 'código processo civil', texto)
    texto = re.sub(r'\bcc/?\d{4}\b', 'código civil', texto)
    texto = re.sub(r'\bctn/?\d{4}\b', 'código tributário nacional', texto)
    texto = re.sub(r'[^\w\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)

    return texto.strip()

In [6]:
def gerar_embedding_local(texto):
    emb = model.encode(texto, convert_to_numpy=True)
    return np.array(emb, dtype=np.float32)

In [7]:
def buscar_sumulas(ementa, k=3, threshold=0.6):
    ementa_proc = preprocessar_texto_juridico(ementa)
    emb_ementa = gerar_embedding_local(ementa_proc)
    emb_ementa = emb_ementa.reshape(1, -1).astype(np.float32)
    
    faiss.normalize_L2(emb_ementa)
    D, I = index.search(emb_ementa, k)
    
    resultados = []
    for sim, idx in zip(D[0], I[0]):
        if sim < threshold:
            continue
        verbete_original = sumulas[idx].get('verbete', '')
        resultados.append({
            'numero': sumulas[idx]['numero'],
            'verbete': verbete_original,
            'similaridade': float(sim)
        })
    return resultados

In [ ]:
ementa_exemplo = """
"""

resposta = buscar_sumulas(ementa_exemplo)
if not resposta:
    print("Nenhum resultado acima do threshold 0.6")
else:
    for i, r in enumerate(resposta, 1):
        print(f"{i}. Súmula {r['numero']} — sim={r['similaridade']:.4f}")
        print(f"{r['verbete']}")
        print()

1. Súmula 676 — sim=0.8365
DIREITO PROCESSUAL PENAL - PRISÃO
Em razão da Lei n. 13.964/2019, não é mais possível ao juiz, de ofício, decretar ou converter prisão em flagrante em prisão preventiva. (TERCEIRA SEÇÃO, julgado em 11/12/2024, DJe de 17/12/2024)

2. Súmula 267 — sim=0.7940
DIREITO PROCESSUAL PENAL - PRISÃO
A interposição de recurso, sem efeito suspensivo, contra decisão condenatória não obsta a expedição de mandado de prisão.
(TERCEIRA SEÇÃO, julgado em 22/05/2002, DJ 29/05/2002, p. 135)

3. Súmula 21 — sim=0.7865
DIREITO PROCESSUAL PENAL - CONSTRANGIMENTO ILEGAL
Pronunciado o réu, fica superada a alegação do constrangimento ilegal da prisão por excesso de prazo na instrução.
(TERCEIRA SEÇÃO, julgado em 06/12/1990, DJ 11/12/1990, p. 14873)

